<div style="position: absolute; top: 0; left: 0; font-family: 'Garamond'; font-size: 16px;">
    <a href="https://github.com/patriciaapenat" style="text-decoration: none; color: inherit;">Patricia Peña Torres</a>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 48px;">
    <strong>Proyecto final, BRFSS-clustering</strong>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 36px;">
    <strong>0.2. Análisis exploratorio</strong>
</div>

__________________

<div style="font-family: 'Garamond'; font-size: 14px;">

En este notebook se llava a cabo lo relativo al análisis exploratorio, por la naturaleza de los datos este EDA se ha centrado principalmente en variables demográficas
    
</div>

<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Configuración del entorno de trabajo</strong>
</div>

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, Imputer
from scipy.spatial import KDTree
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
import pandas as pd
import random
import os.path
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import numpy as np
import warnings
import tensorflow as tf
from pyspark import SparkConf, SparkContext
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from pyspark.ml.linalg import Vectors
from functools import reduce

# Ignorar advertencias deprecated
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# configurar gráficos
sns.set(style="whitegrid", context="notebook", palette="mako")

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Configuración de Spark</strong>
</div>

In [ ]:
# Si hay un SparkContext existente, debemos cerrarlo antes de crear uno nuevo
if 'sc' in locals() and sc:
    sc.stop()  # Detener el SparkContext anterior si existe

# Configuración de Spark
conf = (
    SparkConf()
    .setAppName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .setMaster("local[2]")  # Modo local con un hilo para ejecución
    .set("spark.driver.host", "127.0.0.1")  # Dirección del host del driver
    .set("spark.executor.heartbeatInterval", "3600s")  # Intervalo de latido del executor
    .set("spark.network.timeout", "7200s")  # Tiempo de espera de la red
    .set("spark.executor.memory", "14g")  # Memoria asignada para cada executor
    .set("spark.driver.memory", "14g")  # Memoria asignada para el driver
)

# Crear un nuevo SparkContext con la configuración especificada
sc = SparkContext(conf=conf)

# Configuración de SparkSession (interfaz de alto nivel para trabajar con datos estructurados en Spark)
spark = (
    SparkSession.builder
    .appName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .config("spark.sql.repl.eagerEval.enabled", True)  # Habilitar la evaluación perezosa en Spark SQL REPL
    .config("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Número máximo de filas a mostrar en la evaluación perezosa
    .getOrCreate()  # Obtener la sesión Spark existente o crear una nueva si no existe
) 

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Lectura del archivo</strong>
</div>

In [ ]:
df = spark.read.format("csv").option("header", "true").load(r"C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_imputated_2022.csv")

In [ ]:
# Convertir todas las columnas a tipo numérico
for column_name in df.columns:
    df = df.withColumn(column_name, col(column_name).cast("double"))

# Autoencoder

Un autoencoder es un tipo de red neuronal artificial utilizada en tareas de aprendizaje no supervisado, específicamente en tareas de reducción de dimensionalidad y generación de datos. Su estructura consta de dos partes principales: el codificador (encoder) y el decodificador (decoder). El objetivo principal de un autoencoder es aprender una representación comprimida de los datos de entrada y luego reconstruir los datos originales a partir de esta representación comprimida.

Aquí hay una breve descripción de cada una de las partes de un autoencoder:

1. Codificador (Encoder):
   - La parte del codificador toma los datos de entrada y los transforma en una representación de menor dimensionalidad (también llamada "código" o "embedding").
   - A medida que la red neuronal del codificador reduce la dimensionalidad, está aprendiendo a capturar las características más importantes y relevantes de los datos de entrada.
   - El codificador puede consistir en una o varias capas ocultas, típicamente utilizando funciones de activación como ReLU (Rectified Linear Unit) en cada capa.

2. Decodificador (Decoder):
   - La parte del decodificador toma la representación comprimida del codificador y la expande nuevamente para reconstruir los datos originales.
   - La red del decodificador es esencialmente un espejo inverso del codificador, donde las capas ocultas aumentan gradualmente la dimensionalidad de los datos.
   - El decodificador utiliza una función de activación adecuada en la capa de salida para generar la reconstrucción.

La idea clave detrás de un autoencoder es que la red intenta aprender una representación eficiente de los datos, de modo que la reconstrucción sea lo más cercana posible a los datos originales. El proceso de entrenamiento implica minimizar la diferencia entre los datos de entrada y los datos reconstruidos, lo que fomenta la captura de patrones significativos en los datos.

Los autoencoders tienen diversas aplicaciones, como la reducción de ruido en imágenes, la detección de anomalías, la generación de imágenes sintéticas y la reducción de dimensionalidad para visualización y compresión de datos.

En Keras, puedes implementar un autoencoder utilizando su API de alto nivel, que facilita la construcción y entrenamiento de redes neuronales. Puedes definir un modelo de autoencoder utilizando capas Dense (totalmente conectadas) para el codificador y el decodificador, y luego compilar y entrenar el modelo utilizando datos de entrada y objetivos de reconstrucción.

In [ ]:
columnas_features = [col for col in df.columns if col != "etiqueta"]
ensamblador = VectorAssembler(inputCols=columnas_features, outputCol="features")
df_con_features = ensamblador.transform(df).select("features")

In [ ]:
# Convertir el DataFrame de Spark a un array NumPy
features_array = np.array(df_con_features.rdd.map(lambda x: x.features.toArray()).collect())

In [ ]:
# Definir el autoencoder utilizando TensorFlow
input_dim = len(columnas_features)
encoding_dim = 4  # Dimensión reducida deseada

In [ ]:
# Definir la arquitectura del autoencoder
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l=0.001))(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='relu')(encoder)

In [ ]:
# Crear el modelo del autoencoder
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

Un autoencoder es un tipo de red neuronal que consta de dos partes principales: el codificador $(f(x))$ y el decodificador $(g(z))$. El objetivo principal de un autoencoder es aprender una representación eficiente y comprimida de los datos de entrada, de modo que se puedan reconstruir con la menor pérdida de información posible. Para lograr esto, se utiliza una función de pérdida que mide la diferencia entre los datos de entrada originales $(x)$ y los datos reconstruidos $(x')$.

1. Codificador (Encoder):
   - El codificador toma un vector de entrada $x$ y lo mapea a un vector de representación comprimida $z$ a través de una serie de transformaciones lineales y no lineales. En este caso, el codificador utiliza una activación 'sigmoid' y un término de regularización L2 con $l=0.01$, lo que significa que se aplica una función sigmoide a la salida del codificador y se agrega un término de regularización L2 en la función de pérdida para controlar el sobreajuste:
   $$ z = f(x) $$

2. Decodificador (Decoder):
   - El decodificador toma la representación comprimida $z$ y lo mapea de nuevo al espacio de entrada $x'$ tratando de reconstruir $x$ lo más fielmente posible. En este caso, el decodificador utiliza una activación 'relu':
   $$ x' = g(z) $$

3. Función de Pérdida (Loss Function):
   - Para entrenar el autoencoder, se utiliza la función de pérdida 'categorical_crossentropy', que se usa comúnmente para problemas de clasificación multiclase. En este contexto, se utiliza para medir la discrepancia entre las etiquetas asignadas y las salidas del decodificador. La pérdida 'categorical_crossentropy' se utiliza para evaluar qué tan bien se asignan las etiquetas a las representaciones codificadas:
   $$  {categorical_crossentropy}(y, y') = -\sum_{i} y_i \log(y'_i) $$

   - Donde $y$ son las etiquetas reales y $y'$ son las salidas del decodificador.

4. Entrenamiento:
   - Durante el entrenamiento, el autoencoder busca minimizar la función de pérdida 'categorical_crossentropy', ajustando los parámetros del codificador y el decodificador. Esto se logra utilizando un optimizador Adam con tasas de aprendizaje adaptativas. El término de regularización L2 en el codificador ayuda a controlar el sobreajuste durante el entrenamiento.

El objetivo final es que, después del entrenamiento, el autoencoder aprenda a capturar las características más importantes y relevantes de los datos de entrada en la representación $z$, de modo que la reconstrucción $x'$ sea una versión fiel de $x$, y la pérdida 'categorical_crossentropy' sea mínima. Las representaciones codificadas obtenidas pueden ser útiles en tareas de clasificación o análisis de datos posteriores.

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.99,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
# Entrenar el autoencoder
autoencoder.fit(features_array, features_array, epochs=50, batch_size=32)

In [ ]:
# Obtener las representaciones codificadas de los datos
encoded_features_model = tf.keras.models.Model(inputs=input_layer, outputs=encoder)
encoded_features = encoded_features_model.predict(features_array)

In [ ]:
hi

In [ ]:
# Convertir las representaciones codificadas de vuelta a un DataFrame de Spark
encoded_features_rdd = spark.sparkContext.parallelize(encoded_features.tolist())
encoded_features_df = encoded_features_rdd.map(lambda x: (Vectors.dense(x),)).toDF(["encoded_features"])

In [ ]:
# Entrenar el autoencoder
history = autoencoder.fit(features_array, features_array, epochs=50, batch_size=64)

# Imprimir métricas durante el entrenamiento
print("Métricas durante el entrenamiento:")
print(history.history)

In [ ]:
# Visualizar la pérdida durante el entrenamiento
plt.plot(history.history['loss'])
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.show()

In [ ]:
# Convertir el DataFrame de Spark a un array NumPy
encoded_features_array = np.array(encoded_features_df.select("encoded_features").rdd.map(lambda x: x.encoded_features.toArray()).collect())

# KMEANS

In [ ]:
from sklearn.cluster import KMeans as SKLearnKMeans

In [ ]:
# Crear una lista vacía para almacenar las inercias
cs = []

# Probar diferentes valores de k (número de clusters)
for i in range(1, 20):
    kmeans = SKLearnKMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(encoded_features_array)  # Usar array Numpy de características codificadas

    # Calcular la inercia y añadirla a la lista
    cs.append(kmeans.inertia_)

# Trazar la curva de la inercia en función del número de clusters
plt.plot(range(1, 20), cs, marker='o', linestyle='-', color='blue')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Criterio del Codo')
plt.show()

In [ ]:
# Primero, es necesario asegurarse de que las características estén en formato de vector. 
assembler = VectorAssembler(inputCols=["encoded_features"], outputCol="features")
vectorized_df = assembler.transform(encoded_features_df)

# Aplicar PCA
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")  # k es el número de componentes principales a calcular
model = pca.fit(vectorized_df)
result = model.transform(vectorized_df)

result.show(10)

# DBSCAN


DBSCAN (Density-Based Spatial Clustering of Applications with Noise) es un algoritmo de agrupación (clustering) que se utiliza para identificar grupos de puntos en un conjunto de datos en función de su densidad. A diferencia de otros algoritmos de agrupación como K-Means, DBSCAN no asume que los grupos tienen una forma esférica y puede identificar grupos de diferentes tamaños y formas de manera más flexible.

El algoritmo DBSCAN funciona de la siguiente manera:

1. Selecciona un punto de inicio aleatorio que aún no haya sido visitado ni asignado a ningún grupo.
2. Examina los puntos cercanos a este punto de inicio dentro de un radio especificado llamado "radio epsilon" (ε).
3. Si hay suficientes puntos cercanos dentro de ε, se considera que estos puntos forman un grupo, y se les asigna un etiqueta común.
4. El proceso se repite para todos los puntos dentro del grupo recién formado, y se siguen expandiendo los grupos hasta que no se puedan encontrar más puntos cercanos dentro de ε.
5. Si un punto no puede ser alcanzado por ningún otro punto dentro de ε, se considera un punto "ruido" y no se asigna a ningconsiderados puntos ruido.

DBSCAN es especialmente útil cuando se trabaja con datos en los que los grupos pueden tener formas y tamaños irregulares, y puede ser una alternativa efectiva a otros algoritmos de agrupación más tradicionales como K-Means.ún grupo

In [ ]:
# Escalar los datos para que tengan media 0 y desviación estándar 1
scaler = StandardScaler()
scaled_encoded_features = scaler.fit_transform(encoded_features)

Primero se ajusta el `StandardScaler` a tus datos (`encoded_features_array`) para calcular la media $mu$ y la desviación estándar $sigma$ de cada característica. Luego, transforma los datos restando la media y dividiendo por la desviación estándar para cada característica, resultando en `scaled_encoded_features`, donde cada característica ahora está estandarizada.

In [ ]:
scaled_features_rdd = spark.sparkContext.parallelize(scaled_encoded_features.tolist())

# Convertir el RDD a un DataFrame de Spark
df_scaled = scaled_features_rdd.map(lambda x: (Vectors.dense(x),)).toDF(["features"])

In [ ]:
def find_neighbors(point_idx, points, eps):
    neighbors = []
    point = np.array(points[point_idx])  # Convierte el punto de referencia a un array de Numpy
    for idx, other_point in enumerate(points):
        other_point_array = np.array(other_point)  # Convierte el otro punto a un array de Numpy
        if np.linalg.norm(point - other_point_array) < eps:
            neighbors.append(idx)
    return neighbors

neighbors = find_neighbors(scaled_features_rdd, 5, 5)

In [ ]:
def find_neighbors(point_idx, points, eps):
    neighbors = []
    for idx, other_point in enumerate(points):
        if np.linalg.norm(points[point_idx] - other_point) < eps:
            neighbors.append(idx)
    return neighbors

def expand_cluster(point_idx, neighbors, cluster_id, eps, min_points, points, clusters, visited):
    clusters[point_idx] = cluster_id
    i = 0
    while i < len(neighbors):
        neighbor_idx = neighbors[i]
        if not visited[neighbor_idx]:
            visited[neighbor_idx] = True
            new_neighbors = find_neighbors(neighbor_idx, points, eps)
            if len(new_neighbors) >= min_points:
                neighbors = neighbors + new_neighbors
        if clusters[neighbor_idx] == -1:
            clusters[neighbor_idx] = cluster_id
        i += 1

def dbscan(points, eps, min_points):
    cluster_id = 0
    n_points = len(points)
    clusters = [-1] * n_points
    visited = [False] * n_points
    
    for point_idx in range(n_points):
        if visited[point_idx]:
            continue
        visited[point_idx] = True
        neighbors = find_neighbors(point_idx, points, eps)
        if len(neighbors) < min_points:
            clusters[point_idx] = -1  # Mark as noise
        else:
            expand_cluster(point_idx, neighbors, cluster_id, eps, min_points, points, clusters, visited)
            cluster_id += 1
    return clusters


In [ ]:
# Suponiendo que 'scaled_encoded_features' es un RDD
def apply_dbscan_to_rdd(rdd, eps, min_samples):
    # Esta función necesita ser adaptada para operar en el entorno distribuido de Spark
    local_points = rdd.collect()  # Esto NO es recomendable en práctica por problemas de memoria
    clusters = dbscan(local_points, eps, min_samples)
    return clusters

# Llamar a la función adaptada (esto es solo conceptual)
cluster_labels = apply_dbscan_to_rdd(scaled_features_rdd, 5, 5)




In [ ]:
# Aplicar DBSCAN
dbscan = DBSCAN(eps=5, min_samples=5)
cluster_labels = dbscan.fit_predict(scaled_encoded_features)

In [ ]:
# Añadir las etiquetas de clúster de vuelta al DataFrame de Spark
# Primero, necesitas convertir las etiquetas a un DataFrame de Spark
labels_df = spark.createDataFrame(cluster_labels.tolist(), IntegerType())
labels_df = labels_df.withColumn("row_id", monotonically_increasing_id())

In [ ]:
# Añade un ID a tu DataFrame original para hacer join
df = df.withColumn("row_id", monotonically_increasing_id())

In [ ]:
# Unir las etiquetas de clúster con el DataFrame original
df_with_labels = df.join(labels_df, df.row_id == labels_df.row_id).drop("row_id")

In [ ]:
df_with_labels.show()

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Calcula el Silhouette Score
silhouette_score_value = silhouette_score(scaled_encoded_features, cluster_labels)
print("Silhouette Score:", silhouette_score_value)

# Calcula el Coeficiente de Calinski-Harabasz
calinski_harabasz_score_value = calinski_harabasz_score(scaled_encoded_features, cluster_labels)
print("Calinski-Harabasz Score:", calinski_harabasz_score_value)

# Calcula el Davies-Bouldin Score
davies_bouldin_score_value = davies_bouldin_score(scaled_encoded_features, cluster_labels)
print("Davies-Bouldin Score:", davies_bouldin_score_value)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Suponiendo que `encoded_features_array` contiene las representaciones codificadas de tus datos
# y `cluster_labels` contiene las etiquetas de clúster asignadas por DBSCAN

# Reducir la dimensionalidad con t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_encoded_features = tsne.fit_transform(encoded_features_array)

# Crear un DataFrame de pandas para la visualización
tsne_df = pd.DataFrame(tsne_encoded_features, columns=['Componente 1', 'Componente 2'])
tsne_df['Cluster'] = cluster_labels

# Visualizar los clústeres en un gráfico de dispersión
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Componente 1', y='Componente 2', hue='Cluster', data=tsne_df, palette='mako', legend='full')
plt.title('Visualización de Clústeres con t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend(title='Cluster')
plt.show()


In [ ]:
# Suponiendo que `encoded_features_array` contiene las representaciones codificadas de tus datos
# y `cluster_labels` contiene las etiquetas de clúster asignadas por DBSCAN

# Reducir la dimensionalidad con t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_encoded_features = tsne.fit_transform(encoded_features_array)

# Crear un DataFrame de pandas para la visualización
tsne_df = pd.DataFrame(tsne_encoded_features, columns=['Componente 1', 'Componente 2'])
tsne_df['Cluster'] = cluster_labels

# Filtrar los datos para visualizar solo los cuatro clústeres
tsne_df_filtered = tsne_df[tsne_df['Cluster'] >= 0]  # Solo considerar clústeres con etiquetas positivas

# Visualizar los clústeres en un gráfico de dispersión
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Componente 1', y='Componente 2', hue='Cluster', data=tsne_df_filtered, palette='mako', legend='full')
plt.title('Visualización de Clústeres con t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.legend(title='Cluster')
plt.show()
